In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
import numpy as np
import pandas as pd
data_train = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep = '\t')
data_test = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip', sep = '\t')
data_train.head()
data_train.shape



(156060, 4)

In [32]:
# 提取训练集中的文本内容
train_sentences = data_train['Phrase']

# 提取测试集中的文本内容
test_sentences = data_test['Phrase']

# 构建一个语料库。通过pandas中的contcat函数将训练集和测试集的文本内容合并到一起
sentences = pd.concat([train_sentences, test_sentences])

# 合并的一起的语料库的规模
sentences.shape


(222352,)

In [39]:
# 提取训练集中的情感标签
label = data_train['Sentiment']


In [45]:
from sklearn.feature_extraction.text import CountVectorizer
co = CountVectorizer(
    analyzer = 'word',
    ngram_range=(1,4),
    max_features=15000
)


In [52]:
co.fit(sentences)

CountVectorizer(max_features=15000, ngram_range=(1, 4))

In [55]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_sentences, label,random_state = 1234)
#- x_train 训练集数据 （相当于课后习题）
#- x_test 验证集数据 （相当于模拟考试题）
#- y_train 训练集标签 （相当于课后习题答案）
#- y_test 验证集标签（相当于模拟考试题答案）


In [56]:
x_train = co.transform(x_train)
x_test = co.transform(x_test)
#查看训练集中的一个数据
x_train[1]

<1x15000 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [57]:
from sklearn.linear_model import LogisticRegression
#实例化一个逻辑回归类
lgl = LogisticRegression()
#训练模型
lgl.fit(x_train,y_train)
#预测正确率
print('词袋方法进行文本特征工程，使用sklearn默认的逻辑回归分类器，验证集上的预测正确率',lgl.score(x_test,y_test))

词袋方法进行文本特征工程，使用sklearn默认的逻辑回归分类器，验证集上的预测正确率 0.6467512495194156


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [62]:
from sklearn.model_selection import GridSearchCV
param_grid = { 'C':range(1,10),
              'dual':[True,False]
}

lgGS = LogisticRegression()
grid = GridSearchCV(lgGS, param_grid=param_grid,cv=3, n_jobs=-1)
grid.fit(x_train,y_train)

#最优参数
grid.best_params_

#获取最佳模型
lg_final = grid.best_estimator_
print('经过网格搜索，找到最优超参数组合对应的逻辑回归模型，在验证集上的预测正确率', lg_final.score(x_test,y_test))



/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
27 fits failed out of a total of 54.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
27 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 453, in _check_solver
    "Solver %s supports only dual=False, g

经过网格搜索，找到最优超参数组合对应的逻辑回归模型，在验证集上的预测正确率 0.6467512495194156


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [66]:


# 对测试集中的文本，使用lg_final逻辑回归分类器进行预测
predictions = lg_final.predict

# 查看预测结果
predictions

#将测试结果加在测试集中
data_test.loc[:, 'Sentiment'] = predictions
data_test.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,<bound method LinearClassifierMixin.predict of...
1,156062,8545,An intermittently pleasing but mostly routine ...,<bound method LinearClassifierMixin.predict of...
2,156063,8545,An,<bound method LinearClassifierMixin.predict of...
3,156064,8545,intermittently pleasing but mostly routine effort,<bound method LinearClassifierMixin.predict of...
4,156065,8545,intermittently pleasing but mostly routine,<bound method LinearClassifierMixin.predict of...


In [68]:

#loc通过索引标签来抽取数据：
final_data = data_test.loc[:,['PhraseId','Sentiment']]
final_data.head()

#保存为.csv文件，即为最终结果
final_data.to_csv('final_data.csv',index=None)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
